In [2]:
import glob,os
import torch
from simple_train import load_model
from eval_model import get_image_paths, single_image_inference, infer_folder, evaluate_model

In [3]:
TRAIN_FOLDER = r'D:\Work\Dataset\Misahub-CapsuleVision-Dataset\Dataset\training'
VAL_FOLDER   =   r'D:\Work\Dataset\Misahub-CapsuleVision-Dataset\Dataset\validation'
TEST_FOLDER  = r'D:\Work\Challenges\Misahub-Capsule-Vision\datasets\CapsuleVisionTestDataset\Testing set\Testing set\Images'

train_paths, val_paths, test_image_paths = get_image_paths(TRAIN_FOLDER, 
                                                           VAL_FOLDER,
                                                           TEST_FOLDER)

Number of training folders: 10
Number of validation folders: 10
Number of test images: 4385
Number of training images: 37607
Number of validation images: 16132


In [4]:
best_model = r'D:\Work\Challenges\Misahub-Capsule-Vision\models\efficientvit\ch_checkpoints\22\EfficientViT-L2-bestvalf1-23-0.9065.pt'

model = load_model(num_classes=10)
checkpoint = torch.load(best_model)
model.load_state_dict(checkpoint)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.eval()
model = model.to(device)

print(f'Model path : {best_model}')
print('Model Loaded ..')

df = infer_folder(model,
                  test_image_paths, 
                  type='test',
                  device=device)
    

# === Evaluation model if its training or validation dataset ===
# rep = evaluate_model(df)

# print(rep['classification_report'])  
# print(f'Mean AUC: {rep["mean_auc"]:.2f}')
# print(f'Balanced Accuracy: {rep["balanced_accuracy"]:.2f}')
# print()

Number of trainable parameters: 11447818
Number of non-trainable parameters: 49090208
Model path : D:\Work\Challenges\Misahub-Capsule-Vision\models\efficientvit\ch_checkpoints\22\EfficientViT-L2-bestvalf1-23-0.9065.pt
Model Loaded ..


  0%|          | 0/4385 [00:00<?, ?it/s]c:\Users\girin\anaconda3\envs\torchvision_odd\lib\site-packages\torch\nn\modules\conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ..\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,
100%|██████████| 4385/4385 [02:50<00:00, 25.69it/s]


In [7]:
df.to_excel('test_sub.xlsx',
            index=False)

In [19]:

# single_image_inference(model,
#                        val_paths[300]['image_path'],
#                        device)

In [ ]:
df = infer_folder(model,
                  val_paths, 
                  type='val',
                  device=device)

In [ ]:
df.sample(4)

In [18]:
df.to_csv('model_weighloss_val_preds.csv', index=False)

In [ ]:
rep = evaluate_model(df)

In [ ]:
print(rep['classification_report'])
print(f'Mean AUC: {rep["mean_auc"]:.3f}')
print(f'Balanced Accuracy: {rep["balanced_accuracy"]:.3f}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

class NeuralNetwork:
    def __init__(self, learning_rate=0.01):
        self.learning_rate = learning_rate
        self.weights = np.random.randn(2)  # Initialize weights randomly
    
    def _relu(self, z):
        return np.maximum(z, 0)
    
    def _relu_derivative(self, z):
        return np.where(z > 0, 1, 0)
    
    def forward_propagate(self, X):
        z = np.dot(X, self.weights)
        return self._relu(z)
    
    def backward_propagate(self, X, y, y_pred):
        d_output = 2 * (y_pred - y)
        d_relu = d_output * self._relu_derivative(y_pred)
        d_weights = np.dot(X.T, d_relu)
        return d_weights
    
    def update_weights(self, d_weights):
        self.weights -= self.learning_rate * d_weights
    
    def train(self, X, y, epochs=1000):
        X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
        
        for _ in range(epochs):
            y_pred = self.forward_propagate(X_with_bias)
            d_weights = self.backward_propagate(X_with_bias, y, y_pred)
            self.update_weights(d_weights)
        
        return self.weights
    
    def predict(self, X):
        X_with_bias = np.hstack((np.ones((X.shape[0], 1)), X))
        return self.forward_propagate(X_with_bias)

# Generate sample data
np.random.seed(42)
X = np.random.rand(100, 1)
y = 3 * X + 2 + np.random.randn(100, 1) / 1.5

# Train the model
nn = NeuralNetwork()
weights = nn.train(X, y)

print(f"Final weights: {weights}")

# Make predictions
y_pred = nn.predict(X)

# Evaluate the model
mse = np.mean((y - y_pred) ** 2)
r_squared = 1 - (np.sum((y - y_pred) ** 2) / np.sum((y - np.mean(y)) ** 2))

print(f"Mean Squared Error: {mse:.4f}")
print(f"R-squared: {r_squared:.4f}")

# Visualize the results
plt.scatter(X, y, label='Actual')
plt.plot(X, y_pred, color='red', label='Predicted')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('Linear Regression using Neural Network')
plt.legend()
plt.show()
